In [1]:
#pip install mmcv

In [2]:
#pip install torch torchvision

In [9]:
#pip install deeplake

  Using cached deeplake-3.8.4.tar.gz (574 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached humbug-0.3.2-py3-none-any.whl (15 kB)
  Using cached aioboto3-12.0.0-py3-none-any.whl (32 kB)
  Using cached pathos-0.3.1-py3-none-any.whl (82 kB)
     |████████████████████████████████| 254 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 11.0 MB 133 kB/s eta 0:00:01     |██████████████████████████████▉ | 10.6 MB 133 kB/s eta 0:00:03
     |████████████████████████████████| 135 kB 4.2 MB/s eta 0:00:01
  Using cached aiobotocore-2.7.0-py3-none-any.whl (73 kB)
  Using cached aioitertools-0.11.0-py3-none-any.whl (23 kB)
  Using cached botocore-1.31.64-py3-none-any.whl (11.3 MB)
  Using cached boto3-1.28.64-py3-none-any.whl (135 kB)
  Using cached s3transfer-0.7.0-py3-none-any.whl (79 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached pox-0.3.3-py3-none-any.whl (29 kB)

In [3]:
#import deeplake
import numpy as np

from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models

from mmcv.ops import DeformConv2d

In [10]:
from dataprocessing import celebA_train_dataloader, celebA_val_dataloader, celebA_test_dataloader, adience_dataloader

\

Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/adience



-

...

-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/celeb-a-test



/

hub://activeloop/celeb-a-test loaded successfully.



/Users/tulipmajumder/opt/anaconda3/lib/python3.9/site-packages/deeplake/integrations/pytorch/common.py:126: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


In [15]:
class DeformableGenderClassifier(nn.Module):
    def __init__(self, num_classes):
        super(DeformableGenderClassifier, self).__init__()

        # First Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=7, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.norm1 = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75)

        # Second Convolutional Layer
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.norm2 = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75)

        # Third Convolutional Layer
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=0)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2)

        # Fully Connected Layers
        self.fc1 = nn.Linear(384 * 13 * 13, 512)
        self.relu4 = nn.ReLU()
        self.drop1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 512)
        self.relu5 = nn.ReLU()
        self.drop2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.norm1(self.pool1(self.relu1(self.conv1(x))))
        x = self.norm2(self.pool2(self.relu2(self.conv2(x))))
        x = self.pool3(self.relu3(self.conv3(x)))

        x = x.view(x.size(0), -1)  # Flatten the output for fully connected layers
        x = self.drop1(self.relu4(self.fc1(x)))
        x = self.drop2(self.relu5(self.fc2(x)))
        x = self.fc3(x)

        return x



In [16]:
# Create an instance of the DeformableGenderClassifier
model = DeformableGenderClassifier(num_classes=2)  # Assuming 2 classes: male and female


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
model

DeformableGenderClassifier(
  (conv1): Conv2d(3, 96, kernel_size=(7, 7), stride=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (norm1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (norm2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1))
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64896, out_features=512, bias=True)
  (relu4): ReLU()
  (drop1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (relu5): ReLU()
  (drop2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
)

In [18]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    for inputs, age_labels, gender_labels in celebA_train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        age_loss = criterion(outputs['age'], age_labels)
        gender_loss = criterion(outputs['gender'], gender_labels)
        total_loss = age_loss + gender_loss
        total_loss.backward()
        optimizer.step()

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    for inputs, age_labels, gender_labels in celebA_test_dataloader:
        outputs = model(inputs)
        # Calculate age and gender classification metrics

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x221184 and 64896x512)